In [1]:
import pandas as pd
import ijson
import numpy as np
import jsonlines
import csv
import json

In [41]:
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
summary_tokenizer = AutoTokenizer.from_pretrained("mabrouk/amazon-review-summarizer-bart") #mabrouk/amazon-review-summarizer-bart
summary_model = AutoModelForSeq2SeqLM.from_pretrained("mabrouk/amazon-review-summarizer-bart")
summary_prefix = "summarize: "
def summarize_profile(profile):
    tokens_input = summary_tokenizer.encode(summary_prefix+profile, return_tensors='pt', max_length=512, truncation=True)
    ids = summary_model.generate(tokens_input, max_length=45, min_length=40)
    summary = summary_tokenizer.decode(ids[0], skip_special_tokens=True)
    return summary

In [4]:
from transformers import pipeline
# Load the text classification pipeline
classifier = pipeline('text-classification', model = 'keeyan/nli-roberta-base-finetuned-for-amazon-review-ratings')
def rate_product_review(AIP):
    rating = classifier(AIP)[0]['label']
    if rating == 'terrible':
        return 1
    elif rating == 'poor':
        return 2
    elif rating == 'ok':
        return 3
    elif rating == 'good':
        return 4
    elif rating == 'great':
        return 5
#AIP = "What is the score of the following review on a scale of 1 to 5? just answer with 1, 2, 3, 4, or 5 without further explanation. review: Holy cow.  This case will definitely fill the bill if you're looking for a hard plastic bumper to protect the sides and backs against falls.  However, the rubber skin is really cheap and I can tell will tear easily over time.  Also, this case adds substantial size to your phone.  Feels like I'm carrying a laptop."
#rate_product_review(AIP)

In [3]:
import json
import csv
import ijson
def convert_json_to_csv(json_file_path, csv_file_path, chunk_size=1024):
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["id", "input", "profile_id", "profile_text","PPEP"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        with open(json_file_path, 'r', encoding='utf-8') as jsonfile:
            for entry in ijson.items(jsonfile, 'item'):
                for profile_entry in entry.get("profile", []):
                    writer.writerow({
                        "id": entry.get("id", ""),
                        "input": entry.get("input", ""),
                        "profile_id": profile_entry.get("id", ""),
                        "profile_text": profile_entry.get("text", ""),
                        "PPEP": f"{profile_entry.get('score', '')} is the score for {profile_entry.get('text', '')}"
                    })

# Example usage
json_file_path = 'validation_inputs.json'
csv_file_path = 'validation_inputs.csv'
convert_json_to_csv(json_file_path, csv_file_path)

In [11]:
import ijson
import csv
import pandas as pd

def get_user_aggregated_profiles(input_file, output_file):
    df = pd.read_csv(input_file)

    # Concatenate profile_text for rows with the same user_id
    result_df = df.groupby('id')['PPEP'].agg(lambda x: ' '.join(x)).reset_index()

    result_df.to_csv(output_file, index = False)

    # Display the result
    display(result_df)
    

get_user_aggregated_profiles("validation_inputs.csv", "aggregated_profiles.csv")

,id,PPEP
0,210,4 is the score for I've only had this a couple...
1,211,5 is the score for These fit right into my sto...
2,212,5 is the score for Best beef bone broth I have...
3,213,5 is the score for A quality product and verif...
4,214,5 is the score for Works perfectly and I didn'...
...,...,...
2495,212495,5 is the score for Rcvd replacement phone afte...
2496,212496,5 is the score for Living in Houston in Summer...
2497,212497,4 is the score for These books are a great set...
2498,212498,5 is the score for An idea I first thought was...


In [5]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer

df = pd.read_csv('aggregated_profiles.csv')
sampled_df = df.sample(n=350, random_state=42)
sampled_df = sampled_df.reset_index(drop=True)
display(sampled_df)

def get_similar_user_profiles_RL(user, model_name='all-MiniLM-L6-v2', batch_size=32, k=5, from_sample = False):
    # Load sentence transformer model outside the function for better performance
    sentence_transformer_model = SentenceTransformer(model_name).cuda()

    # Load profiles from CSV
    if from_sample:
        df = sampled_df
    else:
        df = pd.read_csv('aggregated_profiles.csv')

    display(df)

    # Extract user profile
    user_profile = df.loc[df['id'] == user, 'PPEP'].values
    if not user_profile:
        return []  # User not found

    #print('user_profile is \n', user_profile[0])
    user_profile_embedding = torch.Tensor(sentence_transformer_model.encode(user_profile[0])).cuda()

    rl = []

    # Batched similarity calculation for better performance
    for i in range(0, len(df), batch_size):
        batch_profiles = df.loc[i:i+batch_size-1, 'PPEP'].values
        batch_user_ids = df.loc[i:i+batch_size-1, 'id'].values

        # Skip the user in the batch if it's the same as the input user
        mask = batch_user_ids != user
        batch_profiles = batch_profiles[mask]
        batch_user_ids = batch_user_ids[mask]

        # Encode batch profiles on GPU
        batch_profile_embeddings = torch.Tensor(sentence_transformer_model.encode(batch_profiles.tolist())).cuda()


        # Calculate batch similarity scores using dot product on GPU
        batch_similarity_scores = torch.matmul(user_profile_embedding, batch_profile_embeddings.T)

        # Update result list
        rl.extend(zip(batch_user_ids, batch_similarity_scores.cpu().numpy()))

    # Sort the list by similarity score
    rl.sort(key=lambda item: item[1], reverse=True)
    return rl[:k]

# Example usage
#user_id = 210
#similar_users = get_similar_user_profiles_RL(user_id)
#print(similar_users)

,id,PPEP
0,211447,5 is the score for This works excellent with t...
1,211114,"5 is the score for Over 30+ years ago, I playe..."
2,211064,5 is the score for Love this! You can use this...
3,212287,5 is the score for I'll edit this review as ti...
4,211537,5 is the score for I am enjoying having so man...
...,...,...
345,211385,4 is the score for This is the classic distort...
346,21408,1 is the score for Microsoft discontinued the ...
347,211091,5 is the score for I recently bought a Chelsea...
348,212282,5 is the score for still getting used to it. w...


In [23]:
def process_row(row):
    return get_similar_user_profiles_RL(row['id'], k=5, from_sample = True)
def get_top_k_similar_users(output_file):
    sim_users = {}
    df = sampled_df
    df['sim_users'] = df.apply(process_row, axis=1)
    display(df)
    df.to_csv(output_file, index = False)
get_top_k_similar_users('similar_users.csv')

,id,PPEP
0,211447,5 is the score for This works excellent with t...
1,211114,"5 is the score for Over 30+ years ago, I playe..."
2,211064,5 is the score for Love this! You can use this...
3,212287,5 is the score for I'll edit this review as ti...
4,211537,5 is the score for I am enjoying having so man...
...,...,...
345,211385,4 is the score for This is the classic distort...
346,21408,1 is the score for Microsoft discontinued the ...
347,211091,5 is the score for I recently bought a Chelsea...
348,212282,5 is the score for still getting used to it. w...


,id,PPEP
0,211447,5 is the score for This works excellent with t...
1,211114,"5 is the score for Over 30+ years ago, I playe..."
2,211064,5 is the score for Love this! You can use this...
3,212287,5 is the score for I'll edit this review as ti...
4,211537,5 is the score for I am enjoying having so man...
...,...,...
345,211385,4 is the score for This is the classic distort...
346,21408,1 is the score for Microsoft discontinued the ...
347,211091,5 is the score for I recently bought a Chelsea...
348,212282,5 is the score for still getting used to it. w...


,id,PPEP
0,211447,5 is the score for This works excellent with t...
1,211114,"5 is the score for Over 30+ years ago, I playe..."
2,211064,5 is the score for Love this! You can use this...
3,212287,5 is the score for I'll edit this review as ti...
4,211537,5 is the score for I am enjoying having so man...
...,...,...
345,211385,4 is the score for This is the classic distort...
346,21408,1 is the score for Microsoft discontinued the ...
347,211091,5 is the score for I recently bought a Chelsea...
348,212282,5 is the score for still getting used to it. w...


,id,PPEP
0,211447,5 is the score for This works excellent with t...
1,211114,"5 is the score for Over 30+ years ago, I playe..."
2,211064,5 is the score for Love this! You can use this...
3,212287,5 is the score for I'll edit this review as ti...
4,211537,5 is the score for I am enjoying having so man...
...,...,...
345,211385,4 is the score for This is the classic distort...
346,21408,1 is the score for Microsoft discontinued the ...
347,211091,5 is the score for I recently bought a Chelsea...
348,212282,5 is the score for still getting used to it. w...


,id,PPEP
0,211447,5 is the score for This works excellent with t...
1,211114,"5 is the score for Over 30+ years ago, I playe..."
2,211064,5 is the score for Love this! You can use this...
3,212287,5 is the score for I'll edit this review as ti...
4,211537,5 is the score for I am enjoying having so man...
...,...,...
345,211385,4 is the score for This is the classic distort...
346,21408,1 is the score for Microsoft discontinued the ...
347,211091,5 is the score for I recently bought a Chelsea...
348,212282,5 is the score for still getting used to it. w...


...

,id,PPEP
0,211447,5 is the score for This works excellent with t...
1,211114,"5 is the score for Over 30+ years ago, I playe..."
2,211064,5 is the score for Love this! You can use this...
3,212287,5 is the score for I'll edit this review as ti...
4,211537,5 is the score for I am enjoying having so man...
...,...,...
345,211385,4 is the score for This is the classic distort...
346,21408,1 is the score for Microsoft discontinued the ...
347,211091,5 is the score for I recently bought a Chelsea...
348,212282,5 is the score for still getting used to it. w...


,id,PPEP
0,211447,5 is the score for This works excellent with t...
1,211114,"5 is the score for Over 30+ years ago, I playe..."
2,211064,5 is the score for Love this! You can use this...
3,212287,5 is the score for I'll edit this review as ti...
4,211537,5 is the score for I am enjoying having so man...
...,...,...
345,211385,4 is the score for This is the classic distort...
346,21408,1 is the score for Microsoft discontinued the ...
347,211091,5 is the score for I recently bought a Chelsea...
348,212282,5 is the score for still getting used to it. w...


,id,PPEP
0,211447,5 is the score for This works excellent with t...
1,211114,"5 is the score for Over 30+ years ago, I playe..."
2,211064,5 is the score for Love this! You can use this...
3,212287,5 is the score for I'll edit this review as ti...
4,211537,5 is the score for I am enjoying having so man...
...,...,...
345,211385,4 is the score for This is the classic distort...
346,21408,1 is the score for Microsoft discontinued the ...
347,211091,5 is the score for I recently bought a Chelsea...
348,212282,5 is the score for still getting used to it. w...


,id,PPEP
0,211447,5 is the score for This works excellent with t...
1,211114,"5 is the score for Over 30+ years ago, I playe..."
2,211064,5 is the score for Love this! You can use this...
3,212287,5 is the score for I'll edit this review as ti...
4,211537,5 is the score for I am enjoying having so man...
...,...,...
345,211385,4 is the score for This is the classic distort...
346,21408,1 is the score for Microsoft discontinued the ...
347,211091,5 is the score for I recently bought a Chelsea...
348,212282,5 is the score for still getting used to it. w...


,id,PPEP,sim_users
0,211447,5 is the score for This works excellent with t...,"[(21231, 0.71728855), (21387, 0.70657676), (21..."
1,211114,"5 is the score for Over 30+ years ago, I playe...","[(21508, 0.53844863), (2193, 0.5134594), (2147..."
2,211064,5 is the score for Love this! You can use this...,"[(211511, 0.6802315), (21211, 0.6309017), (212..."
3,212287,5 is the score for I'll edit this review as ti...,"[(211124, 0.59207934), (21772, 0.5879262), (21..."
4,211537,5 is the score for I am enjoying having so man...,"[(21857, 0.7073191), (21237, 0.6810317), (2115..."
...,...,...,...
345,211385,4 is the score for This is the classic distort...,"[(212173, 0.47476465), (21942, 0.46741104), (2..."
346,21408,1 is the score for Microsoft discontinued the ...,"[(211944, 0.6295747), (211131, 0.6292778), (21..."
347,211091,5 is the score for I recently bought a Chelsea...,"[(21218, 0.59719545), (211018, 0.56879777), (2..."
348,212282,5 is the score for still getting used to it. w...,"[(211944, 0.5855507), (211131, 0.5711541), (21..."


In [9]:
#function to append query_user_profile with k similar users
import json
import pandas as pd
import ast

def get_appended_user_profiles(query_user_id, no_similar_users = False):
    # Assuming 'get_top_k_similar_users' is a function that returns a list of user tuples (user_id, similarity_score)
    #sim_users = get_similar_user_profiles_RL(user_count, str(query_user_id))

    # Use pandas directly to read and filter the DataFrame
    df = pd.read_csv('validation_inputs.csv')
    sdf = pd.read_csv('similar_users.csv')

    #display(sdf)
    
    query_user_profile = df[df['id'] == int(query_user_id)]
    
    # Extract PPEP column values directly
    query_PPEP = query_user_profile['PPEP'].tolist()

    if no_similar_users:
        print('no similar users')
        return query_PPEP

    sim_user_profiles = []
    is_present = sdf['id'].isin([query_user_id]).any()
    if is_present:
        print("user id present in csv used for training")
        sim_users = sdf.loc[sdf['id'] == query_user_id, 'sim_users'].iloc[0]
        sim_users = ast.literal_eval(sim_users)
        #print("Sim_users from csv: ", sim_users)
        sim_users_profiles = [df[df['id'] == int(sim_user_id)]['PPEP'].tolist() for sim_user_id, _ in sim_users]
    else:
        sim_users = get_similar_user_profiles_RL(query_user_id, k=5, from_sample = False)
        print("Sim_users from aggregated profiles: ", sim_users)
        sim_users_profiles = [df[df['id'] == int(sim_user_id)]['PPEP'].tolist() for sim_user_id, _ in sim_users]

    # Using list comprehension to extract PPEP for similar users
    sim_users_profiles = [df[df['id'] == int(sim_user_id)]['PPEP'].tolist() for sim_user_id, _ in sim_users]

    # Flatten the list of lists
    all_profiles = [query_PPEP] + sim_users_profiles
    all_profiles = [item for sublist in all_profiles for item in sublist]

    #print("ALL PPEP:", all_profiles)
    return all_profiles

#function to train flan t5

In [42]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
retriever_tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
retriever_model = AutoModel.from_pretrained('facebook/contriever')
retriever_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
retriever_model = retriever_model.to(retriever_device)

def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

def AIP(input, query_user_id, no_summary = False, no_sim_users = False):
    retrieved_profiles = retrieve_profiles(input, query_user_id, no_sim_users)
    print(retrieved_profiles)
    aip = ''
    if no_summary:
        print("No Summary")
        for retrieved_profile in retrieved_profiles:
            words = retrieved_profile[0].split()
            if(len(words)>45):
                aip = aip + ' '.join(words[:45]) + ', and '
            else:
                aip = aip + ' '.join(words[:len(words)]) + ', and '
    else:
        for retrieved_profile in retrieved_profiles:
            aip = aip + retrieved_profile[0][:19] + summarize_profile(retrieved_profile[0][19:]) + ', and '
    aip = aip[:-6] + '. ' + input
    print('Final prompt:', aip)
    return aip

def retrieve_profiles(input, query_user_id, no_sim_users, batch_size=32, k=4):
    embeddings_list = []
    split_result = input.split(':', 1)
    query = split_result[1]
    profiles = get_appended_user_profiles(query_user_id, no_sim_users)

    for i in range(0, len(profiles), batch_size):
        batch_profiles = profiles[i:i+batch_size-1]
        batch_profiles.insert(0, query)
        
        batch_profiles_tokens = retriever_tokenizer(batch_profiles, padding=True, truncation=True, return_tensors='pt')
        batch_profiles_tokens = {key: value.to(retriever_device) for key, value in batch_profiles_tokens.items()}

        with torch.no_grad():
            outputs = retriever_model(**batch_profiles_tokens)

        embeddings = mean_pooling(outputs.last_hidden_state, batch_profiles_tokens['attention_mask'])

        query_embedding = embeddings[0]

        # Calculate batch similarity scores using dot product on GPU
        batch_similarity_scores = torch.matmul(query_embedding, embeddings[1:].T)

        embeddings_list.extend(zip(batch_profiles[1:], batch_similarity_scores.cpu().numpy()))

        torch.cuda.empty_cache()
    
    embeddings_list.sort(key=lambda item: item[1], reverse=True)
    return embeddings_list[:k]

#AIP("What is the score of the following review on a scale of 1 to 5? just answer with 1, 2, 3, 4, or 5 without further explanation. review: 10 year old daughter liked this book.  Had allot of info, and was well written for younger kids to understand.", 211447, True, True)

ERROR! Session/line number was not unique in database. History logging moved to new session 58


In [34]:
#RMSE AND MAE
#model_preds
#Creating eval dataset
validation_df_input = pd.read_json('validation_inputs.json')
validation_df_output = pd.read_csv('validation_outputs.csv')
small_eval_dataset = sampled_df.sample(n=30, random_state=42)
small_eval_dataset = small_eval_dataset.reset_index(drop=True)
inputs = []
for index, row in small_eval_dataset.iterrows():
    inputs.append([pd.to_numeric(row['id']), (validation_df_input[validation_df_input['id'] == row['id']]['input'].iloc[0]), validation_df_output[validation_df_output['id'] == row['id']]['output'].iloc[0]])
print(inputs)

[[211598, 'What is the score of the following review on a scale of 1 to 5? just answer with 1, 2, 3, 4, or 5 without further explanation. review: It was not what I expected but it was a great read! I really really like it.\nI liked the characters. And the story was well paced.', 5], [21679, "What is the score of the following review on a scale of 1 to 5? just answer with 1, 2, 3, 4, or 5 without further explanation. review: I'm really irked with this machine. It ran great the few times i've used it, but now will not start. An internet search shows many others having similar problems. i thought the Husqvarna brand was supposed to make quality tools; up there with Honda and Stihl. Ah-well live and learn.", 1], [21199, 'What is the score of the following review on a scale of 1 to 5? just answer with 1, 2, 3, 4, or 5 without further explanation. review: This is a good history of the 29th Infantry Division in the Normandy campaign.  It covers the training of the division in the United State

In [43]:
preds_no_summary_no_samples = []
for input in inputs:
    preds_no_summary_no_samples.append(rate_product_review(AIP(input[1],input[0], True, True)))
print(preds_no_summary_no_samples)

no similar users
[('5 is the score for I really enjoyed this one. I only wished that Daniel had a POV would\nhave love to know what went through his mind!\nBut overall the story was well paced the characters where great!', 1.1770658), ('5 is the score for I really enjoyed this one. I only wished that Daniel had a POV would\nhave love to know what went through his mind!\nBut overall the story was well paced the characters where great!', 1.1770658), ("5 is the score for It was not what I expected. There where slow Z (zombies) and fast Z.\nFast Z's where changing into something else than the slow zombies I know off.\nBut I really liked it and read it in one sitting. Because I wanted to know with who Claire\nended! I hope book 2 will come soon......", 1.1465284), ('5 is the score for A great story! I also loved the romance that is in it.\nOverall I really enjoyed it.', 1.1457993)]
No Summary
Final prompt: 5 is the score for I really enjoyed this one. I only wished that Daniel had a POV wou

RuntimeError: The expanded size of the tensor (787) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 787].  Tensor sizes: [1, 514]

In [44]:
print(len(preds_no_summary_no_samples))

30


In [45]:
print(preds_no_summary_no_samples)

[4, 2, 4, 5, 5, 2, 5, 5, 5, 3, 3, 4, 5, 4, 5, 5, 5, 4, 4, 1, 2, 2, 4, 5, 3, 5, 4, 5, 4, 4]


In [47]:
preds_no_summary_similar = []
for input in inputs[:30]:
    preds_no_summary_similar.append(rate_product_review(AIP(input[1],input[0], True, False)))
print(preds_no_summary_similar)

user id present in csv used for training
[('5 is the score for I really enjoyed this one. I only wished that Daniel had a POV would\nhave love to know what went through his mind!\nBut overall the story was well paced the characters where great!', 1.1770658), ('5 is the score for I really enjoyed this one. I only wished that Daniel had a POV would\nhave love to know what went through his mind!\nBut overall the story was well paced the characters where great!', 1.1770658), ("5 is the score for It was not what I expected. There where slow Z (zombies) and fast Z.\nFast Z's where changing into something else than the slow zombies I know off.\nBut I really liked it and read it in one sitting. Because I wanted to know with who Claire\nended! I hope book 2 will come soon......", 1.1465284), ('5 is the score for A great story! I also loved the romance that is in it.\nOverall I really enjoyed it.', 1.1457993)]
No Summary
Final prompt: 5 is the score for I really enjoyed this one. I only wished t

In [55]:
truth_labels = []
for input in inputs[:30]:
    truth_labels.append(input[2])
print(truth_labels)

[5, 1, 4, 4, 5, 2, 5, 5, 5, 3, 3, 5, 5, 3, 5, 5, 5, 4, 4, 5, 1, 3, 5, 4, 1, 5, 4, 5, 4, 5]


In [52]:
preds_summary_similar = []
for input in inputs[:30]:
    preds_summary_similar.append(rate_product_review(AIP(input[1],input[0], False, False)))
print(preds_summary_similar)

user id present in csv used for training
[('5 is the score for I really enjoyed this one. I only wished that Daniel had a POV would\nhave love to know what went through his mind!\nBut overall the story was well paced the characters where great!', 1.1770658), ('5 is the score for I really enjoyed this one. I only wished that Daniel had a POV would\nhave love to know what went through his mind!\nBut overall the story was well paced the characters where great!', 1.1770658), ("5 is the score for It was not what I expected. There where slow Z (zombies) and fast Z.\nFast Z's where changing into something else than the slow zombies I know off.\nBut I really liked it and read it in one sitting. Because I wanted to know with who Claire\nended! I hope book 2 will come soon......", 1.1465284), ('5 is the score for A great story! I also loved the romance that is in it.\nOverall I really enjoyed it.', 1.1457993)]
Final prompt: 5 is the score for I really enjoyed this one. I only wished that Daniel 

In [53]:
preds_summary_no_similar = []
for input in inputs[:30]:
    preds_summary_no_similar.append(rate_product_review(AIP(input[1],input[0], False, True)))
print(preds_summary_no_similar)

no similar users
[('5 is the score for I really enjoyed this one. I only wished that Daniel had a POV would\nhave love to know what went through his mind!\nBut overall the story was well paced the characters where great!', 1.1770658), ('5 is the score for I really enjoyed this one. I only wished that Daniel had a POV would\nhave love to know what went through his mind!\nBut overall the story was well paced the characters where great!', 1.1770658), ("5 is the score for It was not what I expected. There where slow Z (zombies) and fast Z.\nFast Z's where changing into something else than the slow zombies I know off.\nBut I really liked it and read it in one sitting. Because I wanted to know with who Claire\nended! I hope book 2 will come soon......", 1.1465284), ('5 is the score for A great story! I also loved the romance that is in it.\nOverall I really enjoyed it.', 1.1457993)]
Final prompt: 5 is the score for I really enjoyed this one. I only wished that Daniel had a POV would have lov

In [56]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

MAE_no_similar_no_summary = mean_absolute_error(y_true = truth_labels, y_pred = preds_no_summary_no_samples)
RMSE_no_similar_no_summary = mean_squared_error(y_true = truth_labels , y_pred = preds_no_summary_no_samples, squared = False)

MAE_similar_no_summary = mean_absolute_error(y_true = truth_labels, y_pred = preds_no_summary_similar)
RMSE_similar_no_summary = mean_squared_error(y_true = truth_labels , y_pred = preds_no_summary_similar, squared = False)

MAE_no_similar_summary = mean_absolute_error(y_true = truth_labels, y_pred = preds_summary_no_similar)
RMSE_no_similar_summary = mean_squared_error(y_true = truth_labels, y_pred = preds_summary_no_similar, squared = False)

MAE_similar_summary = mean_absolute_error(y_true = truth_labels, y_pred = preds_summary_similar)
RMSE_similar_summary = mean_squared_error(y_true = truth_labels , y_pred = preds_summary_similar, squared = False)

print('MAE_no_similar_no_summary', MAE_no_similar_no_summary)
print('RMSE_no_similar_no_summary',RMSE_no_similar_no_summary)
print('MAE_similar_no_summary',MAE_similar_no_summary)
print('RMSE_similar_no_summary',RMSE_similar_no_summary)
print('MAE_no_similar_summary',MAE_no_similar_summary)
print('RMSE_no_similar_summary',RMSE_no_similar_summary)
print('MAE_similar_summary',MAE_similar_summary)
print('RMSE_similar_summary',RMSE_similar_summary)

MAE_no_similar_no_summary 0.5333333333333333
RMSE_no_similar_no_summary 1.0
MAE_similar_no_summary 0.6333333333333333
RMSE_similar_no_summary 1.0801234497346435
MAE_no_similar_summary 0.43333333333333335
RMSE_no_similar_summary 0.7071067811865476
MAE_similar_summary 0.6333333333333333
RMSE_similar_summary 0.9128709291752769


In [ ]:
!unzip "./final_model.zip" -d "./model_loc"

In [ ]:
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

last_checkpoint = "./model_loc/model_path"

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

In [ ]:
def generate_paraphrase(llm_input):
  tokenized_input = tokenizer(llm_input, return_tensors="pt")
  outputs = finetuned_model.generate(**tokenized_input,max_length = 500)
  answer = tokenizer.decode(outputs[0])
  print("printing answer", type(answer))
  return answer


In [ ]:
# print(len(inputs))
preds_with_summary_with_samples = []
preds_without_summary_without_samples = []
preds_with_summary_without_samples = []
preds_without_summary_with_samples = []

# print(inputs)
ground_truth = []
for input in inputs:
    ground_truth.append(input[2])
    aip_output = AIP(input[1],input[0], True, True)
    print("printing aip output", aip_output)
    preds_without_summary_without_samples.append(generate_paraphrase(aip_output))
    print("printing 1 ",preds_without_summary_without_samples)
    preds_with_summary_with_samples.append(generate_paraphrase(AIP(input[1],input[0], False, False)))
    print("printing 2 ",preds_with_summary_with_samples)
    preds_without_summary_with_samples.append(generate_paraphrase(AIP(input[1],input[0], True, False)))
    print("printing 3 ",preds_without_summary_with_samples)
    preds_with_summary_without_samples.append(generate_paraphrase(AIP(input[1],input[0], False, True)))
    print("printing 4 ",preds_with_summary_without_samples)




# print(preds_with_summary_with_samples)


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np

def calculate_rouge(reference_array, candidate_array):
    # Tokenize reference sentences
    reference_tokens = [sentence.split() for sentence in reference_array]
    
    # Calculate ROUGE-1 (unigram overlap) score
    rouge1_scores = [sentence_bleu(reference_tokens, candidate.split(), weights=(1, 0, 0, 0), smoothing_function=SmoothingFunction().method1) for candidate in candidate_array]

    # Calculate ROUGE-L (longest common subsequence) score
    rougeL_scores = [sentence_bleu(reference_tokens, candidate.split(), weights=(0, 0, 1, 0), smoothing_function=SmoothingFunction().method1) for candidate in candidate_array]

    return rouge1_scores, rougeL_scores

# Example arrays of sentences and ground truth
df = pd.read_csv("your_output_file_transposed.csv")
# print(df.head())

# Assuming your columns are named "ground_truth", "prediction_1", "prediction_2", "prediction_3"
preds_with_summary_with_samples = df["pred1"].tolist()
preds_without_summary_without_samples = df["pred2"].tolist()
preds_with_summary_without_samples = df["pred3"].tolist()
preds_without_summary_with_samples = df["pred4"].tolist()
ground_truth = df["truth"].tolist()

# print(len(ground_truth))

# Calculate ROUGE scores for each array of sentences
rouge1_scores_1, rougeL_scores_1 = calculate_rouge(ground_truth, preds_with_summary_with_samples)
rouge1_scores_2, rougeL_scores_2 = calculate_rouge(ground_truth, preds_without_summary_without_samples)
rouge1_scores_3, rougeL_scores_3 = calculate_rouge(ground_truth, preds_with_summary_without_samples)
rouge1_scores_4, rougeL_scores_4 = calculate_rouge(ground_truth, preds_without_summary_with_samples)

# Print the scores
print('ROUGE-1 Scores Array 1:', np.mean(rouge1_scores_1))
print('ROUGE-L Scores Array 1:', np.mean(rougeL_scores_1))
print('ROUGE-1 Scores Array 2:', np.mean(rouge1_scores_2))
print('ROUGE-L Scores Array 2:', np.mean(rougeL_scores_2))
print('ROUGE-1 Scores Array 3:', np.mean(rouge1_scores_3))
print('ROUGE-L Scores Array 3:', np.mean(rougeL_scores_3))
print('ROUGE-1 Scores Array 4:', np.mean(rouge1_scores_4))
print('ROUGE-L Scores Array 4:', np.mean(rougeL_scores_4))
